In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer

from sklearn.feature_extraction.text import CountVectorizer

Using TensorFlow backend.


In [2]:
reviews = pd.read_csv('employee_reviews.csv')

In [3]:
print (reviews)


       Unnamed: 0    company           location          dates  \
0               1     google               none   Dec 11, 2018   
1               2     google  Mountain View, CA   Jun 21, 2013   
2               3     google       New York, NY   May 10, 2014   
3               4     google  Mountain View, CA    Feb 8, 2015   
4               5     google    Los Angeles, CA   Jul 19, 2018   
5               6     google  Mountain View, CA    Dec 9, 2018   
6               7     google       New York, NY   Dec 11, 2018   
7               8     google               none   Dec 11, 2018   
8               9     google       New York, NY   Dec 10, 2018   
9              10     google               none    Dec 9, 2018   
10             11     google  Mountain View, CA    Dec 8, 2018   
11             12     google               none    Dec 8, 2018   
12             13     google               none    Dec 4, 2018   
13             14     google  San Francisco, CA    Dec 3, 2018   
14        

In [4]:
reviews.head()

,Unnamed: 0,company,location,dates,job-title,summary,pros,cons,advice-to-mgmt,overall-ratings,work-balance-stars,culture-values-stars,carrer-opportunities-stars,comp-benefit-stars,senior-mangemnet-stars,helpful-count,link
0,1,google,none,"Dec 11, 2018",Current Employee - Anonymous Employee,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,none,5.0,4.0,5.0,5.0,4.0,5.0,0,https://www.glassdoor.com/Reviews/Google-Revie...
1,2,google,"Mountain View, CA","Jun 21, 2013",Former Employee - Program Manager,"Moving at the speed of light, burn out is inev...","1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,1) Don't dismiss emotional intelligence and ad...,4.0,2.0,3.0,3.0,5.0,3.0,2094,https://www.glassdoor.com/Reviews/Google-Revie...
2,3,google,"New York, NY","May 10, 2014",Current Employee - Software Engineer III,Great balance between big-company security and...,"* If you're a software engineer, you're among ...","* It *is* becoming larger, and with it comes g...",Keep the focus on the user. Everything else wi...,5.0,5.0,4.0,5.0,5.0,4.0,949,https://www.glassdoor.com/Reviews/Google-Revie...
3,4,google,"Mountain View, CA","Feb 8, 2015",Current Employee - Anonymous Employee,The best place I've worked and also the most d...,You can't find a more well-regarded company th...,I live in SF so the commute can take between 1...,Keep on NOT micromanaging - that is a huge ben...,5.0,2.0,5.0,5.0,4.0,5.0,498,https://www.glassdoor.com/Reviews/Google-Revie...
4,5,google,"Los Angeles, CA","Jul 19, 2018",Former Employee - Software Engineer,"Unique, one of a kind dream job",Google is a world of its own. At every other c...,"If you don't work in MTV (HQ), you will be giv...",Promote managers into management for their man...,5.0,5.0,5.0,5.0,5.0,5.0,49,https://www.glassdoor.com/Reviews/Google-Revie...


In [5]:
reviews.shape


(67529, 17)

In [6]:
reviews.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67529 entries, 0 to 67528
Data columns (total 17 columns):
Unnamed: 0                    67529 non-null int64
company                       67529 non-null object
location                      67529 non-null object
dates                         67529 non-null object
job-title                     67529 non-null object
summary                       67409 non-null object
pros                          67529 non-null object
cons                          67529 non-null object
advice-to-mgmt                67232 non-null object
overall-ratings               67529 non-null float64
work-balance-stars            67529 non-null object
culture-values-stars          67529 non-null object
carrer-opportunities-stars    67529 non-null object
comp-benefit-stars            67529 non-null object
senior-mangemnet-stars        67529 non-null object
helpful-count                 67529 non-null int64
link                          67529 non-null object
dtypes: 

In [7]:
reviews.describe()


,Unnamed: 0,overall-ratings,helpful-count
count,67529.000000,67529.000000,67529.000000
mean,33765.000000,3.826075,1.268211
std,19494.087501,1.154989,16.085852
min,1.000000,1.000000,0.000000
25%,16883.000000,3.000000,0.000000
50%,33765.000000,4.000000,0.000000
75%,50647.000000,5.000000,1.000000
max,67529.000000,5.000000,2094.000000


In [8]:
reviews['procons'] = reviews['pros'] + " " + reviews['cons']

In [9]:
reviews[2:2]

,Unnamed: 0,company,location,dates,job-title,summary,pros,cons,advice-to-mgmt,overall-ratings,work-balance-stars,culture-values-stars,carrer-opportunities-stars,comp-benefit-stars,senior-mangemnet-stars,helpful-count,link,procons


In [10]:
reviews.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67529 entries, 0 to 67528
Data columns (total 18 columns):
Unnamed: 0                    67529 non-null int64
company                       67529 non-null object
location                      67529 non-null object
dates                         67529 non-null object
job-title                     67529 non-null object
summary                       67409 non-null object
pros                          67529 non-null object
cons                          67529 non-null object
advice-to-mgmt                67232 non-null object
overall-ratings               67529 non-null float64
work-balance-stars            67529 non-null object
culture-values-stars          67529 non-null object
carrer-opportunities-stars    67529 non-null object
comp-benefit-stars            67529 non-null object
senior-mangemnet-stars        67529 non-null object
helpful-count                 67529 non-null int64
link                          67529 non-null object
procons 

In [11]:
reviews['text length'] = reviews['procons'].apply(len)
reviews.head()

#this creates a new column that will apply the text length of review

,Unnamed: 0,company,location,dates,job-title,summary,pros,cons,advice-to-mgmt,overall-ratings,work-balance-stars,culture-values-stars,carrer-opportunities-stars,comp-benefit-stars,senior-mangemnet-stars,helpful-count,link,procons,text length
0,1,google,none,"Dec 11, 2018",Current Employee - Anonymous Employee,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,none,5.0,4.0,5.0,5.0,4.0,5.0,0,https://www.glassdoor.com/Reviews/Google-Revie...,People are smart and friendly Bureaucracy is s...,64
1,2,google,"Mountain View, CA","Jun 21, 2013",Former Employee - Program Manager,"Moving at the speed of light, burn out is inev...","1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,1) Don't dismiss emotional intelligence and ad...,4.0,2.0,3.0,3.0,5.0,3.0,2094,https://www.glassdoor.com/Reviews/Google-Revie...,"1) Food, food, food. 15+ cafes on main campus ...",3452
2,3,google,"New York, NY","May 10, 2014",Current Employee - Software Engineer III,Great balance between big-company security and...,"* If you're a software engineer, you're among ...","* It *is* becoming larger, and with it comes g...",Keep the focus on the user. Everything else wi...,5.0,5.0,4.0,5.0,5.0,4.0,949,https://www.glassdoor.com/Reviews/Google-Revie...,"* If you're a software engineer, you're among ...",4699
3,4,google,"Mountain View, CA","Feb 8, 2015",Current Employee - Anonymous Employee,The best place I've worked and also the most d...,You can't find a more well-regarded company th...,I live in SF so the commute can take between 1...,Keep on NOT micromanaging - that is a huge ben...,5.0,2.0,5.0,5.0,4.0,5.0,498,https://www.glassdoor.com/Reviews/Google-Revie...,You can't find a more well-regarded company th...,4187
4,5,google,"Los Angeles, CA","Jul 19, 2018",Former Employee - Software Engineer,"Unique, one of a kind dream job",Google is a world of its own. At every other c...,"If you don't work in MTV (HQ), you will be giv...",Promote managers into management for their man...,5.0,5.0,5.0,5.0,5.0,5.0,49,https://www.glassdoor.com/Reviews/Google-Revie...,Google is a world of its own. At every other c...,12870


In [12]:
X = reviews['procons']
y = reviews['overall-ratings']

In [13]:
print(type(X))

<class 'pandas.core.series.Series'>


In [14]:
# tokenizer = Tokenizer(num_words=2000)
# tokenizer.fit_on_texts(X)
# sequences = tokenizer.texts_to_sequences(X)
# one_hot_results = tokenizer.texts_to_matrix(X, mode='binary')
# word_index = tokenizer.word_index

# print('Found %s unique tokens.' % len(word_index))
# print(tokenizer)
# print(one_hot_results)
# print(word_index)


In [15]:
import re 
def getwordlist(text):
    # Convert words to lower case and split them
    text = text.lower().split()    
    text = " ".join(text)    
    #Remove Special Characters
    text=re.sub(r'[^a-z\d ]','',text)    
    #Replace Numbers
    text=re.sub(r'\d+','n',text)
    # Return a list of words
    return(text)

comments = []
for text in X:
    comments.append(getwordlist(text))

In [16]:
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(comments) 
sequences = tokenizer.texts_to_sequences(comments)


word_index = tokenizer.word_index
print('unique words %s' % len(word_index))

data = pad_sequences(sequences, maxlen=2500, padding='post')
print('Train data shape:', data.shape)



unique words 45666
Train data shape: (67529, 2500)


In [17]:
# test_comments = []
# for t in text:
#     test_comments.append(getwordlist(t))
    
# test_sequences = tokenizer.texts_to_sequences(test_comments)
# test_data = pad_sequences(test_sequences, maxlen=2500, padding='post')
# print('Test data shape:', test_data.shape)

In [18]:
 from sklearn.preprocessing import LabelBinarizer
 encoder = LabelBinarizer()
 transfomed_label = encoder.fit_transform(y)

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, transfomed_label, test_size=0.2, random_state=42)

In [20]:
#Glove Vectors loaded

embeddings_index = {}

f = open('glove.6B.100d.txt', encoding="utf8")

for line in f:

    values = line.split()

    word = values[0]

    coefs = np.asarray(values[1:], dtype='float32')

    embeddings_index[word] = coefs

f.close()

 

print('Total %s word vectors.' % len(embeddings_index))

Total 400000 word vectors.


In [21]:
#do not need to do pipelining

#want to use the model.fit- parse the train data and then specify the epochs and batch size.


# prepare embeddings for layer
from keras.layers.embeddings import Embedding

words_max = len(word_index)+1

embedding_matrix = np.zeros((len(word_index)+1, 100))

for word, i in word_index.items():

    if i >= words_max:

        continue

    embedding_vector = embeddings_index.get(word)

    if embedding_vector is not None:

        embedding_matrix[i] = embedding_vector

In [22]:
##this is where add Karuna Code
# Create an non re-trainable embedding layer with embedding size 100

nb_words = len(word_index)+1

embedding_layer = Embedding(nb_words,100,

        weights=[embedding_matrix],

        input_length=2500,

        trainable=True)

In [23]:
from keras.layers import Embedding, LSTM, Dense, Dropout, Lambda, Flatten
from keras.models import Sequential, load_model, model_from_config
import keras.backend as K

def get_model():
    """Define the model."""
    model = Sequential()
    model.add(embedding_layer)
    model.add(LSTM(64, dropout=0.7, recurrent_dropout=0.7))
 
    #model.add(LSTM(300, dropout=0.4, recurrent_dropout=0.4, input_shape=[1, 300], return_sequences=True))
   # model.add(LSTM(64, recurrent_dropout=0.4))
    #model.add(Dropout(0.5))
    model.add(Dense(5, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
    model.summary()

    return model

## Sets up the model for how much data is processed at once.

In [24]:
model = get_model()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2500, 100)         4566700   
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
Total params: 4,609,265
Trainable params: 4,609,265
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#x is the comment, or reviews, can be many columns,  y is the labels

In [ ]:
#This is the training you need to save the data to train
hist = model.fit(X_train, y_train, 
        epochs=7, batch_size=40, validation_split=0.2)


Instructions for updating:
Use tf.cast instead.
Train on 43218 samples, validate on 10805 samples
Epoch 1/7
30680/43218 [====================>.........] - ETA: 3:28:54 - loss: 1.4242 - acc: 0.3371

In [ ]:
#loss = model.evaluate(X_test, y_test, verbose=1)
#print('Accuracy: %f' % (accuracy*100))
accr = model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

In [ ]:
text = "this is a test to see if it works, Karuna is the best!!!"

In [ ]:
prediction = model.predict(test_data)

In [ ]:
from sklearn.metrics import accuracy_score
prediction = model.predict(X_test)
prediction = np.argmax(prediction, axis = 1)
accuracy_score(y_test, prediction)